In [ ]:
import random
import math

import numpy as np

In [ ]:
class Particle:
    def __init__(self, items, vmax, smin, smax, eval_func, max_weight = 65):
        self.items = items
        self.vmax = vmax
        self.smin = smin
        self.smax = smax
        
        self.values = np.array([random.uniform(0, vmax) for _ in items])
        self.speeds = np.array([random.uniform(smin, smax) for _ in items])

        self.best = self.values
        
        self.eval_func = eval_func
        
        self.max_weight = max_weight

    @property
    def fitness(self):
        return self.evaluate(self.values)

    @property
    def fitness_best(self):
        return self.evaluate(self.best)
    
    def evaluate(self, values):
        rv = values.round()

        p = sum(rv * self.items[:, 0])
        w = sum(rv * self.items[:, 1])
        
        params = [p, w] if w <= self.max_weight else [0, self.max_weight * 10]
        
        return (self.eval_func(params), params)
    
    def update(self, g_best, w, ap, ag):
        speed_range = lambda v: min(self.smax, max(self.smin, v))
        value_range = lambda v: min(self.vmax, max(0, v))

        new_speeds = self.speeds * w + ((self.best - self.values) * random.uniform(0, ap)) + ((g_best - self.values) * random.uniform(0, ag))

        self.speeds = np.array([speed_range(s) for s in new_speeds])

        new_values = self.values + self.speeds
        self.values = np.array([value_range(v) for v in new_values])
        
        if self.evaluate(self.values)[0] > self.evaluate(self.best)[0]:
            self.best = self.values

In [ ]:
def population(items, n, vmax, smin, smax):
    eval_func = lambda x: sum(np.array(x) * [1, -1])
    
    return [Particle(items, vmax, smin, smax, eval_func) for _ in range(n)]

In [ ]:
def optimize(items, n, qty, speed, w = 0.8, ap = 0.7, ag = 0.7, ngen = 20):
    ps = population(items, n, qty, -speed, speed)
    
    best_particle = lambda: ps[np.argmax([p.fitness_best[0] for p in ps])]
    
    for i in range(ngen):
        g_best = best_particle().best

        for p in ps:
            p.update(g_best, w, ap, ag)

    return best_particle()

In [ ]:
def print_particle(p):
    print(f'values:{p.values.round()}, fitness:{p.fitness}, speeds:{p.speeds}')
    print(f'best:{p.best.round()}, best_fitness:{p.fitness_best}')   

In [ ]:
items1 = np.array([
    [120, 10],
    [130, 12],
    [80, 7],
    [100, 9],
    [250, 21],
    [185, 16]
])

In [ ]:
print_particle( optimize(items1, 300, 3, 3) )

In [ ]:
print_particle( optimize(items1, 100, 4, 4) )

In [ ]:
import collections

collections.Counter([tuple(optimize(items1, 100, 3, 3).best.round()) for _ in range(50)])